# Test of DiffVis

## Settings

In [1]:
import functools
import html
import pathlib
from pprint import pprint
import re
import sys
import unicodedata

In [2]:
import MeCab
from IPython.display import HTML

In [3]:
sys.path.append('../')
from DiffVis.diffvis import DiffVis

## Define

In [4]:
def make_template(source, target, edit_history):
    template = []
    blank = '<blank>'
    i, j = 0, 0
    for operation in edit_history:
        if operation == 'match':
            template.append(source[i])
            i += 1
            j += 1
        elif operation == 'replace':
            template.append(blank)
            i += 1
            j += 1
        elif operation == 'delete':
            template.append(blank)            
            i += 1
        elif operation == 'insert':
            template.append(blank)
            j += 1

    if not template:
        return template
    
    # delete duplicates
    template_new = [template[0]]
    for i in range(1, len(template)):
        elem_now = template[i]
        elem_last = template[i-1]
        if (elem_now == blank) and (elem_last == blank):
            continue
        else:
            template_new.append(elem_now)
    template = template_new
    return template

In [5]:
class Tokenizer(object):
    """Tokenize Japanese sentence to words.

    Args:
        mecab_dict_path (str or pathlib.Path): Path to MeCab dictionary.
        parts_of_speech (list[str]): List that contains part of speeches to be extracted.
            If None (default), these are used:
                [
                    '名詞', '動詞', '形容詞', '副詞', '助詞',
                    '接続詞', '助動詞', '連体詞', '感動詞',
                ]
        normalize (bool): Flag to determine whether to transform to base form or not.
            Defaults to False.

    Attributes:
        tagger (MeCab.Tagger): Tagger.
        parts_of_speech (list[str]): List that contains part of speeches to be extracted.
        normalize (bool): Flag to determine whether to transform to base form or not.
    """
    def __init__(self, mecab_dict_path='', parts_of_speech=None, normalize=False,):
        added = ''
        if mecab_dict_path:
            added = ' -d' + str(mecab_dict_path)
        self.tagger = MeCab.Tagger('-Ochasen' + added)
        self.tagger.parse('')
        if parts_of_speech:
            self.parts_of_speech = parts_of_speech
        else:
            self.parts_of_speech = [
                '名詞', '動詞', '形容詞', '副詞', '助詞',
                '接続詞', '助動詞', '連体詞', '感動詞',
                ]
        self.normalize = normalize

    def tokenize(self, sentence):
        """Tokenize Japanese sentence to words.

        Args:
            sentence (str): Sentence to tokenize.

        Returns:
            words (list[str]): Extracted words.
        """
        res = self.tagger.parseToNode(sentence)
        words = []
        if self.normalize:
            while res:
                content = res.feature.split(',')
                part_of_speech = content[0]
                if part_of_speech in self.parts_of_speech:
                    word = content[6]
                    if word == '*':
                        word = res.surface
                    words.append(word)
                res = res.next
        else:
            while res:
                word = res.surface
                part_of_speech = res.feature.split(',')[0]
                if part_of_speech in self.parts_of_speech:
                    words.append(word)
                res = res.next
        return words

## Toy Data

### Simple string

In [6]:
source = 'すももも桃も桃のうち'
target = 'すもももももももものうち'
m, n = len(source), len(target)
print(m, n)

10 12


In [7]:
dv = DiffVis(source, target)
dv.build()

In [8]:
print(dv.distance(normalize=True))
print(dv.distance(normalize=False))

0.3333333333333333
4


In [9]:
dv.cost_table

((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12),
 (1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11),
 (2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10),
 (3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9),
 (4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8),
 (5, 4, 3, 2, 1, 1, 2, 3, 4, 5, 6, 7, 8),
 (6, 5, 4, 3, 2, 1, 1, 2, 3, 4, 5, 6, 7),
 (7, 6, 5, 4, 3, 2, 2, 2, 3, 4, 5, 6, 7),
 (8, 7, 6, 5, 4, 3, 3, 3, 3, 4, 4, 5, 6),
 (9, 8, 7, 6, 5, 4, 4, 4, 4, 4, 5, 4, 5),
 (10, 9, 8, 7, 6, 5, 5, 5, 5, 5, 5, 5, 4))

In [10]:
dv.edit_history

('match',
 'match',
 'match',
 'match',
 'replace',
 'match',
 'replace',
 'insert',
 'insert',
 'match',
 'match',
 'match')

In [11]:
print(dv.generate_comparison(mode='console'))

すももも桃も桃  のうち
すもももももももものうち


In [12]:
HTML(dv.generate_comparison(mode='html'))

In [13]:
HTML(dv.generate_comparison(mode='htmltab'))

す,も,も,も,桃,も,桃,,,の,う,ち
す,も,も,も,も,も,も,も,も,の,う,ち


In [14]:
make_template(source, target, dv.edit_history)

['す', 'も', 'も', 'も', '<blank>', 'も', '<blank>', 'の', 'う', 'ち']

### Tokenize

In [15]:
tokenizer = Tokenizer()

In [16]:
source = 'すももも桃も桃のうち'
target = 'すもももももももものうち'
m, n = len(source), len(target)
print(m, n)

10 12


In [17]:
source = tokenizer.tokenize(source)
target = tokenizer.tokenize(target)
m, n = len(source), len(target)
print(m, n)

7 7


In [18]:
dv = DiffVis(source, target)
dv.build()

In [19]:
print(dv.distance(normalize=True))
print(dv.distance(normalize=False))

0.2857142857142857
2


In [20]:
dv.cost_table

((0, 1, 2, 3, 4, 5, 6, 7),
 (1, 0, 1, 2, 3, 4, 5, 6),
 (2, 1, 0, 1, 2, 3, 4, 5),
 (3, 2, 1, 1, 2, 3, 4, 5),
 (4, 3, 2, 2, 1, 2, 3, 4),
 (5, 4, 3, 3, 2, 2, 3, 4),
 (6, 5, 4, 4, 3, 3, 2, 3),
 (7, 6, 5, 5, 4, 4, 3, 2))

In [21]:
dv.edit_history

('match', 'match', 'replace', 'match', 'replace', 'match', 'match')

In [22]:
print(dv.generate_comparison(mode='console'))

すももも桃も桃のうち
すもももももももものうち


In [23]:
HTML(dv.generate_comparison(mode='html'))

In [24]:
HTML(dv.generate_comparison(mode='htmltab'))

すもも,も,桃,も,桃,の,うち
すもも,も,もも,も,もも,の,うち


In [25]:
make_template(source, target, dv.edit_history)

['すもも', 'も', '<blank>', 'も', '<blank>', 'の', 'うち']